# sales_2023_sexage EDA 플랜

| 단계 | 내용 | 상세 작업 |
|:---|:---|:---|
| 1 | 기본 구조 확인 | - 데이터 크기(shape) 확인<br>- 컬럼 목록(columns) 확인<br>- 데이터 타입(dtypes) 확인<br>- 데이터 샘플(head, tail) 확인 |
| 2 | 결측치 확인 | - 컬럼별 결측치 개수 확인(isnull.sum()) |
| 3 | 주요 지표 요약 | - 총 매출건수 합계(매출건수)<br>- 총 매출금액 합계(매출금액합계) |
| 4 | 주요 범주 분석 | - 업종별(분류) 매출 합계<br>- 성별(성별) 매출 합계<br>- 연령대별(연령대) 매출 합계<br>- 지역별(시군구명) 매출 합계 |
| 5 | 시각화 | - 업종별 매출 합계 Top 10 (barplot)<br>- 연령대별 매출 합계 (lineplot)<br>- 성별 매출 합계 (barplot)<br>- 업종별 평균단가 Top 10 (barplot)<br>- 지역별 매출 합계 Top 10 (barplot) |
| 6 | 특이사항 탐색 (선택) | - 업종별 매출 단가 이상 여부<br>- 지역별 특정 연령대 소비 집중 여부 |


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

path = '../data_raw/'
sales_2023_sexage = pd.read_csv(path + 'card_sales_2023_sexage.csv', encoding='cp949')


FileNotFoundError: [Errno 2] No such file or directory: '../data_raw/card_sales_2023_sexage.csv'

In [ ]:
print("Shape:", sales_2023_sexage.shape)

In [ ]:
print("Columns:", sales_2023_sexage.columns)

In [ ]:
sales_2023_sexage.dtypes

In [ ]:
sales_2023_sexage.head()

In [ ]:
sales_2023_sexage.tail()

In [ ]:
sales_2023_sexage.isnull().sum()

In [ ]:
print("총 매출 건수:", sales_2023_sexage['매출건수'].sum())

In [ ]:
print("총 매출 금액:", sales_2023_sexage['매출금액합계'].sum())

In [ ]:
print("업종 분포")
print(sales_2023_sexage['분류'].value_counts())

In [ ]:
print("\n성별 분포")
print(sales_2023_sexage['성별'].value_counts())

In [ ]:
print("\n연령대 분포")
print(sales_2023_sexage['연령대'].value_counts())

In [ ]:
top10_category = (sales_2023_sexage
                  .groupby('분류')['매출금액합계']
                  .sum()
                  .sort_values(ascending=False)
                  .head(10))

plt.figure(figsize=(12,6))
sns.barplot(x=top10_category.values, y=top10_category.index)
plt.title('업종별 매출 합계 Top 10')
plt.xlabel('총 매출금액(원)')
plt.ylabel('업종')
plt.show()

In [ ]:
age_sales = (sales_2023_sexage
             .groupby('연령대')['매출금액합계']
             .sum()
             .sort_index())

plt.figure(figsize=(10,5))
sns.lineplot(x=age_sales.index, y=age_sales.values, marker='o')
plt.title('연령대별 매출 합계')
plt.xlabel('연령대')
plt.ylabel('총 매출금액(원)')
plt.show()


In [ ]:
sex_sales = (sales_2023_sexage
             .groupby('성별')['매출금액합계']
             .sum()
             .sort_values(ascending=False))

plt.figure(figsize=(6,4))
sns.barplot(x=sex_sales.index, y=sex_sales.values)
plt.title('성별 매출 합계')
plt.xlabel('성별')
plt.ylabel('총 매출금액(원)')
plt.show()

In [ ]:
category_avg_price = (sales_2023_sexage
                      .groupby('분류')
                      .agg({'매출금액합계': 'sum', '매출건수': 'sum'}))

category_avg_price['평균단가'] = category_avg_price['매출금액합계'] / category_avg_price['매출건수']

top10_avg_price = category_avg_price['평균단가'].sort_values(ascending=False).head(10)

plt.figure(figsize=(12,6))
sns.barplot(x=top10_avg_price.values, y=top10_avg_price.index)
plt.title('업종별 평균단가 Top 10')
plt.xlabel('평균 결제 금액(원)')
plt.ylabel('업종')
plt.show()

In [ ]:
region_sales = (sales_2023_sexage
                .groupby('시군구명')['매출금액합계']
                .sum()
                .sort_values(ascending=False)
                .head(10))

plt.figure(figsize=(12,6))
sns.barplot(x=region_sales.values, y=region_sales.index)
plt.title('지역별 매출 합계 Top 10')
plt.xlabel('총 매출금액(원)')
plt.ylabel('지역명')
plt.show()


In [ ]:
region_age_sales = (sales_2023_sexage
                    .groupby(['시군구명', '연령대'])['매출금액합계']
                    .sum()
                    .reset_index())

pivot_table = region_age_sales.pivot(index='시군구명', columns='연령대', values='매출금액합계')

plt.figure(figsize=(15,10))
sns.heatmap(pivot_table, cmap='YlGnBu')
plt.title('지역별-연령대별 매출 합계 Heatmap')
plt.show()
